In [ ]:
%%capture
!pip install langchain_google_genai langchain langchain-community
!pip install langchain chromadb
!pip install langchain_huggingface faiss-cpu
!pip install langchain-text-splitters
!pip install pypdf

In [ ]:
from dotenv import load_dotenv
from google.colab import userdata
import os

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import PromptTemplate

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter
)

from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint
from langchain_community.vectorstores import FAISS, Chroma


In [ ]:
# Get and set API token
HUGGINGFACEHUB_API_TOKEN = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = HUGGINGFACEHUB_API_TOKEN


In [ ]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')
print(model.invoke('what is 2 +3 ').content)

2 + 3 = 5


In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
# text = " Nepal is country "

# result = embeddings.embed_query(text)
# print(result)

# load vector data

In [ ]:
loaded_vector_store = FAISS.load_local('faiss_index', embeddings, allow_dangerous_deserialization=True)

In [ ]:
print("Total vectors:", loaded_vector_store.index.ntotal)

Total vectors: 523


In [ ]:
retriever = loaded_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})
print("Retriever created from the loaded vector store.")

Retriever created from the loaded vector store.


In [ ]:
# query = "what is photosynthesis"
# retrieved_docs = retriever.invoke(query)
# print(retrieved_docs)

In [ ]:
prompt = PromptTemplate(
    template="""
You are a helpful AI assistant acting strictly as a legal advocate for matters related only to Nepal.
Use ONLY the information provided in the context. Do not rely on outside knowledge.

Your tasks:
- Answer the question based solely on the context.
- Identify applicable actions, charges, penalties, or fines under Nepali law **only if they are present in the context**.
- If the context does not contain enough information, respond briefly with: "Insufficient information in the context."

Context:
{context}

Question:
{question}

Provide a clear, concise, and context-grounded answer.
""",
    input_variables=["context", "question"]
)


In [ ]:
question          = "My Husband demand for  dowry "
retrieved_docs    = retriever.invoke(question)

In [ ]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"Section 132. \nFor the offence of Forced Labor in Marriage: Compelling a spouse to work without compensation, the \npunishment is Up to 5 years imprisonment and a fine of up to NPR 50,000 according to the section of \nConstitution of Nepal, Article 29. \nFor the offence of Illegal Confinement: Unlawfully restricting a woman's freedom, the punishment is Up \nto 3 years imprisonment and a fine of up to NPR 30,000 according to the section of Muluki Ain (General \nCode), Chapter on Harassment, Section 223. \nFor the offence of Forced Migration: Compelling a woman to migrate against her will, the punishment is \nUp to 10 years imprisonment according to the section of Constitution of Nepal, Article 45. \nFor the offence of Violation of Reproductive Rights: Denying a woman control over her reproductive \nchoices, the punishment is Legal action to restore rights according to the section of Constitution of \nNepal, Article 38.\n\nthe contingency f und into which such moneys as may be determine

In [ ]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [ ]:
answer = model.invoke(final_prompt)
print(answer.content)

Under Nepali law, demanding dowry is a prohibited harmful social practice.

**Applicable Actions/Charges:**
*   Harmful Practices (specifically, dowry)

**Penalties:**
The context states that "fines and imprisonment" are provided for those who engage in or encourage such practices. However, the specific amounts for fines or duration for imprisonment related to dowry are not detailed in the provided information.
